In [1]:
import requests
import pandas as pd
import json
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
!pip install pandasql
from pandasql import sqldf

 

You should consider upgrading via the 'e:\ian\ebay_project\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
#accepts a teams name as input, team name must have spaces between words and can be partial name
#Retrieve team_id and team_link, prints matched team to verify correct match

def get_team_id(team_name):
    _link = requests.get('https://statsapi.web.nhl.com/api/v1/teams').json()
    _team = pd.DataFrame.from_dict(_link['teams'])
    team_name = team_name.title()
    for x in _team['name']:
        if team_name in x:
            id = _team[_team['name'] == x]['id']
            link = _team[_team['name'] == x]['link']
            team_name_final = x
            return id.values[0],link.values[0],x
            
        else:
            pass


In [3]:
def get_link(what,y1 = None, y2 = None):
    base_link  = 'https://statsapi.web.nhl.com'
    
    if y1 is not None and y2 is not None:
        link = requests.get(f'{base_link}{what}&season={y1}{y2}').json()
        
    else:
        link = requests.get(f'{base_link}{what}').json()
    return link

In [4]:
def create_panthers_player_boxscores_table(team_name,start,end):
    
    team_id,team_link,_ = get_team_id(team_name)
    
    
    data = {'game_id' : []
            ,'id' : []
            ,'fullName' : []
        }

    season_games = f'/api/v1/schedule?teamId={team_id}&'
    extention = '/boxscore'

    ind = 0
    while start+1 <= end:
        
        start_link = get_link(season_games,start,start+1)

        for day in start_link['dates']:
            #if multiple games on a date
            for game in day['games']:                
            
                game_id = game['gamePk']
            
                link = f'/api/v1/game/{game_id}{extention}'
                
                game_link = get_link(f'{link}')
                
                if game_link['teams']['away']['team']['id'] == team_id:
                    side = 'away'
                else:
                    side = 'home'
                
                for x in game_link['teams'][f'{side}']['players'].keys():
                    
                    temp = game_link['teams'][f'{side}']['players'][x]
                    if not temp['stats']:
                        pass
                    else:
                        
                        data['game_id'].append(game_id)
                        data['fullName'].append(temp['person']['fullName'])
                
                        data['id'].append(temp['person']['id'])
                        
                        _stats = temp['stats'].keys()
                        for _stat in _stats: 
                        #add new column to dataframe if the stat has not been seen before and append nans to all previous rows
                            for key in temp['stats'][_stat].keys():
                                if key not in data.keys():
                                    data[key] = []
                                    if ind == 0:
                                        pass
                                    else:
                                        for t in range(0,ind):
                                
                                            data[key].append(np.nan) 
                                            
                                    data[key].append(temp['stats'][_stat][f'{key}'])
                                else:
                                    data[key].append(temp['stats'][_stat][f'{key}'])
                                    
                            for w in data.keys():
                                
                                if w in temp['stats'][_stat].keys() or w in['id','fullName', 'game_id']:
                                    pass
                                else:
                                    data[w].append(np.nan)
        
                        ind += 1 
        start +=1       
    df = pd.DataFrame(data)         
    return df

In [5]:
def create_panthers_game_boxscores_table(team_name,start,end):
    team_id,team_link,_ = get_team_id(team_name)
    
    season_games = f'/api/v1/schedule?teamId={team_id}&'
    
    
    data = {'game_id' : []
            ,f'{team_name}_where': []
            ,'result' : []
            ,f'{team_name}_score' : []
            ,'opponent_score' : []
            ,f'{team_name}_shotsOnGoal' :[]
            ,'opponent_shotsOnGoal' :[]
            ,'season' : []
            }

   
    extention = '/linescore'

    ind = 0
    
    link_array = []
    while start+1 <= end:
        
        p = get_link(season_games,start,start+1)
        
        for x in p['dates']:
            for x in x['games']:
                
            
                g_id = x['gamePk']
            
                link = f'/api/v1/game/{g_id}{extention}'
        
                d = get_link(link)
                
                if d['teams']['away']['team']['id'] == team_id:
                    side = 'away'
                    oside = 'home'
                else:
                    side = 'home'
                    oside = 'away'
                _us = d['teams'][f'{side}']
                _them = d['teams'][f'{oside}']
                _our_goals = _us['goals']
                _their_goals = _them['goals']
                data['game_id'].append(g_id)
                data[f'{team_name}_where'].append(side)
                data['season'].append(f'{start}-{start+1}')
                
                data[f'{team_name}_score'].append(_our_goals)
                data['opponent_score'].append(_their_goals)
                data[f'{team_name}_shotsOnGoal'].append(_us['shotsOnGoal'])
                data['opponent_shotsOnGoal'].append(_them['shotsOnGoal'])
                if _our_goals > _their_goals:
                    data['result'].append('win')
                elif d['currentPeriodOrdinal'] == 'OT' and _our_goals < _their_goals:
                    data['result'].append('overtime loss')
                else:
                    data['result'].append('loss')
        start+=1 
    df = pd.DataFrame(data)
    
    return df

In [6]:
def create_panthers_players_table(team_name,start,end):
    team_ext = '/roster?expand=team.roster&'
    team_id,team_link,team_name = get_team_id(team_name)
    team_link = f'{team_link}{team_ext}'
    def yearsago(years, from_date=None):
        if from_date is None:
            from_date = datetime.now()
        return from_date - relativedelta(years=years)

    def num_years(begin, end=None):
        if end is None:
            pass
        num_years = int((end - begin).days / 365.2425)
        num_days = int((end - begin).days % 365.2425)
        if begin > yearsago(num_years, end):
           return f'{num_years-1}-{num_days}d'
        else:
            return f'{num_years}-{num_days}d'
    
    
    data = {
            'id' : []
            ,'fullName' : []
            ,'season' : []
            ,'age' : []
            ,'weight' : []
            ,'height' : []
        }
    

    extention = '/stats?stats=yearByYear'
    ind = 0
    id_name_dict = {}
    end_val = ((end)*10000)+start+1
    start_val = ((start-1)*10000)+start
    while start+1 <= end:
        
        start_link = get_link(team_link,start,start+1)
        

        for x in start_link['roster']:
            
            id_name_dict[x['person']['id']] = x['person']['fullName']
            
        start+=1
    
    

    
    for id,name in id_name_dict.items():
        link_personal = f'/api/v1/people/{id}'
        link = f'/api/v1/people/{id}/{extention}'
        json_personal = get_link(link_personal)
        #print(t_personal)
        json_stats = get_link(link)
        
        for x in json_stats['stats'][0]['splits']:
            if x['team']['name'] == team_name and int(x['season']) < end_val and int(x['season']) > start_val:
                
                _year = x['season'][4:]
                _birth = json_personal['people'][0]['birthDate']
                age = num_years(datetime.strptime(_birth,'%Y-%m-%d'),datetime.strptime(f'{_year}-01-31','%Y-%m-%d'))
                
                temp_data = x['stat']
                data['id'].append(id)
                data['fullName'].append(name)
                data['season'].append(x['season'][0:4]+' - '+x['season'][4:])
                data['age'].append(age)

                data['weight'].append(json_personal['people'][0]['weight'])
                data['height'].append(json_personal['people'][0]['height'])
                
                  
                for key in temp_data.keys():
                    if key not in data.keys():
                        data[key] = []
                        if ind == 0:
                            pass
                        else:
                            for _temp in range(0,ind):
                        
                                data[key].append(np.nan)
                        data[key].append(temp_data[f'{key}'])
                    else:
                        data[key].append(temp_data[f'{key}'])
                for w in data.keys():
                    if w in temp_data.keys() or w in ['id','fullName','season','age','weight','height']:
                        pass
                    else:
                        data[w].append(np.nan)
                ind += 1        
            
    
        

                    
    new_data  = pd.DataFrame(data)        
        
            
    return new_data

In [7]:
start_season = 2014
end_season = 2022
df1 = create_panthers_players_table('panthers',start_season,end_season)
df2 = create_panthers_game_boxscores_table('panthers',start_season,end_season)
df3 = create_panthers_player_boxscores_table('panthers',start_season,end_season)

root_folder = 'E:\Ian\Panthers_project'
df1.to_csv(f'{root_folder}\players_dataframe.csv', encoding='utf-8', index=False)
df2.to_csv(f'{root_folder}\\boxscore_dataframe.csv', encoding='utf-8', index=False)
df3.to_csv(f'{root_folder}\player_boxscore_dataframe.csv', encoding='utf-8', index=False)

In [8]:
pysqldf = lambda q: sqldf(q, globals())

In [9]:
q = '''
SELECT fullName, COUNT(*) as seasons_played
FROM df1
GROUP BY fullName
ORDER BY seasons_played DESC
LIMIT 5

'''
q1_answer = pysqldf(q)
print(q1_answer)

             fullName  seasons_played
0  Jonathan Huberdeau               8
1   Aleksander Barkov               8
2        Aaron Ekblad               8
3    Vincent Trocheck               6
4    MacKenzie Weegar               6


In [10]:
q = '''
WITH cte AS (
    SELECT 
        game_id
        , fullname
        , plusminus
    FROM df3
    WHERE plusMinus = (
        SELECT 
            MAX(plusMinus)
        FROM df3))

SELECT 
    game_id
    , fullname
    , result
FROM cte

LEFT JOIN df2
USING(game_id)


'''
q2_answer = pysqldf(q)
print(q2_answer)

      game_id         fullname result
0  2021020661  Anthony Duclair    win
1  2021020793     Sam Reinhart    win
2  2021020793  Mason Marchment    win
3  2021020793    Anton Lundell    win


In [11]:
q = '''
WITH 
cte AS( 
        SELECT 
                min(age)
        FROM df1
        WHERE id IN (

                SELECT id 
                FROM df1
                WHERE season = '2021 - 2022'))
,cte_id AS (
        SELECT id, fullname from df1
        WHERE age = (SELECT * from cte)
        )
SELECT *
FROM df3
WHERE id IN (
        SELECT id 
        FROM cte_id)
LIMIT 5


'''
q3_answer = pysqldf(q)
print(q3_answer)

      game_id       id      fullName timeOnIce  assists  goals   pim  shots  \
0  2017010017  8480015  Owen Tippett     14:56        1      0  None      3   
1  2017010078  8480015  Owen Tippett     14:22        0      0  None      1   
2  2017010092  8480015  Owen Tippett     13:17        0      0  None      2   
3  2017010093  8480015  Owen Tippett      9:59        0      0  None      1   
4  2017020083  8480015  Owen Tippett     11:39        0      0  None      7   

  saves powerPlaySaves  ... giveaways shortHandedGoals shortHandedAssists  \
0  None           None  ...       0.0              0.0                0.0   
1  None           None  ...       0.0              0.0                0.0   
2  None           None  ...       0.0              0.0                0.0   
3  None           None  ...       1.0              0.0                0.0   
4  None           None  ...       0.0              0.0                0.0   

  blocked plusMinus evenTimeOnIce powerPlayTimeOnIce shortHand

In [12]:
q = '''
WITH 
cte_max_minutes AS (
        SELECT 
                MAX(penaltyMinutes)
        FROM df1
        )
SELECT 
        fullname
        ,penaltyMinutes 
        ,season
FROM df1
WHERE penaltyMinutes = (
        SELECT * 
        FROM cte_max_minutes
        )
'''
q4_answer = pysqldf(q)
print(q4_answer)

             fullName penaltyMinutes       season
0  Alexander Petrovic             98  2017 - 2018


In [13]:
q = '''
SELECT 
    season
    , COUNT(*) as home_shutouts
    , RANK() OVER(ORDER BY COUNT(*)) as rank
FROM df2
WHERE opponent_score = 0 AND panthers_where = 'home'
GROUP BY season
ORDER BY rank DESC
'''
q5_answer = pysqldf(q)
print(q5_answer)

      season  home_shutouts  rank
0  2017-2018              4     8
1  2015-2016              3     6
2  2020-2021              3     6
3  2016-2017              2     3
4  2019-2020              2     3
5  2021-2022              2     3
6  2014-2015              1     1
7  2018-2019              1     1
